### Scraping Lyrics from genius.com

In [1]:
# define API token
GENIUS_API_TOKEN='4fatfe8KqnJEFgO6HIU1KT_BjNBau9CGNbI-by4aQVeDSDMmfZYb6SyMqmPMVcfv'

In [2]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

In [3]:
# function to request artist info
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# function to generate two lists of song names and song urls
def request_song_info(artist_name, song_cap):
    page = 1
    song_names = []
    song_urls = []
    
    while len(song_names) < song_cap:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URLs and names from song objects
        for song in song_info:
            if len(song_names) < song_cap:
                url = song['result']['url']
                name = song['result']['title']
                song_names.append(name)
                song_urls.append(url)
            
        page += 1
        
    print('Found {} songs by {}'.format(len(song_names), artist_name))
    return song_names, song_urls


In [4]:
# Function to scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics_div = html.find('div', id ='lyrics-root')
    if lyrics_div:
        lyrics_text = lyrics_div.get_text(separator='\n')
        # remove anything in between brackets
        lyrics_text = re.sub(r'\n', '#', lyrics_text)
        lyrics_text = re.sub(r'\[.+?\]', '', lyrics_text)
        lyrics_text = re.sub(r'#', '\n', lyrics_text)
        # remove parentheses
        lyrics_text = re.sub(r'\(','', lyrics_text)
        lyrics_text = re.sub(r'\)','', lyrics_text)
        # remove "embed" line at the end
        lyrics_text = re.sub('Embed', '', lyrics_text)
        # remove a number if it's the last line of the lyrics
        lyrics_text = re.sub(r'\d+$', '', lyrics_text.strip())
        # remove empty lines
        lyrics_text = os.linesep.join([s for s in lyrics_text.splitlines() if s])
        # Find the index of the first appearance of the word "lyrics"
        start_index = lyrics_text.lower().find('lyrics')
        if start_index != -1:
            # Exclude the word "lyrics" from the output
            start_index += len('lyrics')
            lyrics_text = lyrics_text[start_index:].strip()
        return lyrics_text
    else:
        return "Lyrics not found on the page."

In [5]:
# function to write a song's lyrics to a file
def write_song_to_file(artist_name, song_name, song_url):
    f = open('corpus_comparative/' + song_name + '_' + artist_name + '.txt', 'wb') # change file path depending on where you want it to end up
    lyrics = scrape_song_lyrics(song_url)
    f.write(lyrics.encode("utf8"))
    f.close()

In [7]:
# Execute the following code for any given artist

artist_name = input("Enter the artist's name: ")
num_songs = int(input("Enter the number of songs to retrieve: "))
song_names, song_urls = request_song_info(artist_name, num_songs)

# DEMO
print(scrape_song_lyrics(song_urls[1]))

# write files
for i in range(0, len(song_names)-1):
    write_song_to_file(artist_name, song_names[i], song_urls[i])


Found 3 songs by Maluma
This is the remix
Son las doce, tú me dices si algarete nos vamo'
Tú sabe' mejor después que quemamo'
Dame una señal y nos quedamos
Haciéndolo tú y yo bien rico
Una onza de krippy, yeh
Contigo la paso happy como hippie
Bebecita, es que me la pones fácil
Siempre esta' mojá', tú debes ser Piscis Baby
Te dejé en la casa después de hacerlo, mami
Mami
; después de hacerlo
Pregunta la niña Niña: "
3 A.M., ¿qué hace en línea, daddy?
"
¿Qué más, pues? Más pues
Sigo aquí pensando en que En que
Sigo aquí pensando en que
En qué rico fue, en qué rico fue
En qué rico, en qué rico
¿Qué más, pues?
Sigo aquí pensando en que En que
Sigo aquí pensando en que
En qué rico fue, en qué rico fue
En qué rico, en qué rico
You might also like
Pri, yah-yah-yah; ¡Farru!
Ella me escribió después que la guayé ¡Blep!
Porque me vio en línea preguntó si ya llegué Yah-yah-yah
Que si me gustó, ella quiere saber
Porque tiene curiosidad y quiere volverlo a hacer Jajaja
Que la llame por la cámara po

In [5]:
lyrics_test = '''
[Letra de "Danza Kuduro" ft. Lucenzo]

[Intro: Don Omar]
A&X
El Orfanato
¡Danza Kuduro!
(Plop, plop, plop, plop, plop, plop, plop)
¡Lucenzo!
(El Orfanato)
¡El Rey!

[Coro: Don Omar]
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro

[Verso: Don Omar]
¿Quién puede domar la fuerza del mal
Que se mete por tu' vena'? (¡Don!)
Lo caliente del sol que se te metió
Y no te deja quieta, nena (¡Toma, toma!)
¿Quién puede parar eso que al bailar
Descontrola tu' cadera'? (Sexy!)
Y ese fuego que quema por dentro y lento
Te convierte en fiera (Rah)

[Coro: Don Omar]
Con la mano arriba, cintura sola
Da media vuelta y sacude duro
No te quite' ahora que esto solo empieza
Mueve la cabeza y sacude duro (¡Don!)

[Refrán: Lucenzo]
Mexe kuduro
Balança que é uma loucura
Morena, vem a meu lado
Ninguém vai ficar parado
Quero ver mexer kuduro
Balança que é uma loucura
Morena, vem a meu lado
Ninguém vai ficar parado, oh-oh-oh

[Pre-Coro: Lucenzo]
Oi, oi, oi
Oi, oi, oi, oi
É pa' quebrar kuduro
Vamos dançar kuduro
Oi, oi, oi
Oi, oi, oi, oi
Seja morena ou loira
Vem balançar kuduro
Oi, oi, oi

[Coro: Don Omar]
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro

[Refrán: Lucenzo]
Balança que é uma loucura
Morena, vem a meu lado
Ninguém vai ficar parado
Quero ver mexer kuduro
Balança que é uma loucura
Morena, vem a meu lado
Ninguém vai ficar parado

[Pre-Coro: Lucenzo & Don Omar]
Oi, oi, oi
Oi, oi, oi, oi
É pa' quebrar kuduro
Vamos dançar kuduro
Oi, oi, oi
Oi, oi, oi, oi
Seja morena ou loira
Vem balançar kuduro
Oi, oi, oi
¡El Orfanato!

[Coro: Don Omar]
La mano arriba (Vai, vai; vuelta), cintura sola (Vai, vai; vuelta)
Da media vuelta (Vai, vai; vuelta), danza kuduro (Vai, vai; vuelta)
No te canse' ahora (Vai, vai; vuelta) que esto solo empieza (Vai, vai; vuelta)
Mueve la cabeza (Vai, vai; vuelta), danza kuduro (Vai, vai; vuelta)
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro

[Outro: Don Omar]
A&X (A&X)
'''

lyrics_test = re.sub(r'\n', '#', lyrics_test)
lyrics_test = re.sub(r'\[.+?\]', '', lyrics_test)
lyrics_test = re.sub(r'#', '\n', lyrics_test)
lyrics_test = os.linesep.join([s for s in lyrics_test.splitlines() if s])
# Find the index of the first appearance of the word "lyrics"


print(lyrics_test)

A&X
El Orfanato
¡Danza Kuduro!
(Plop, plop, plop, plop, plop, plop, plop)
¡Lucenzo!
(El Orfanato)
¡El Rey!
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro
La mano arriba, cintura sola
Da media vuelta, danza kuduro
No te canse' ahora que esto solo empieza
Mueve la cabeza, danza kuduro
¿Quién puede domar la fuerza del mal
Que se mete por tu' vena'? (¡Don!)
Lo caliente del sol que se te metió
Y no te deja quieta, nena (¡Toma, toma!)
¿Quién puede parar eso que al bailar
Descontrola tu' cadera'? (Sexy!)
Y ese fuego que quema por dentro y lento
Te convierte en fiera (Rah)
Con la mano arriba, cintura sola
Da media vuelta y sacude duro
No te quite' ahora que esto solo empieza
Mueve la cabeza y sacude duro (¡Don!)
Mexe kuduro
Balança que é uma loucura
Morena, vem a meu lado
Ninguém vai ficar parado
Quero ver mexer kuduro
Balança que é uma loucura
Morena, vem a meu lado
Ninguém vai ficar parado, oh-oh-oh
Oi, oi, oi